In [1]:
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for creating regular densely-connected NN layer.

# Data manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Sklearn
import sklearn # for model evaluation
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics

# Other utilities
import sys
import os

# Assign main directory to a variable
main_dir=os.path.dirname(sys.path[0])
#print(main_dir)

In [2]:
import pandas as pd
# Set Pandas options to display more columns
pd.options.display.max_columns=50

# Read in the weather data csv
df=pd.read_csv(main_dir+'/data/weatherHistory.csv', encoding='utf-8')


In [3]:
df

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 19:00:00.000 +0200,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
96449,2016-09-09 20:00:00.000 +0200,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
96450,2016-09-09 21:00:00.000 +0200,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
96451,2016-09-09 22:00:00.000 +0200,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


In [4]:
df.isnull().count()

Formatted Date              96453
Summary                     96453
Precip Type                 96453
Temperature (C)             96453
Apparent Temperature (C)    96453
Humidity                    96453
Wind Speed (km/h)           96453
Wind Bearing (degrees)      96453
Visibility (km)             96453
Loud Cover                  96453
Pressure (millibars)        96453
Daily Summary               96453
dtype: int64

In [5]:
# Drop records where target RainTomorrow=NaN
df.drop("Formatted Date", axis=1, inplace=True)
df.drop("Daily Summary", axis=1, inplace=True)

In [6]:
# For other columns with missing values, fill them in with column mean
#df=df.fillna(df.mean())

In [7]:
df.dtypes

Summary                      object
Precip Type                  object
Temperature (C)             float64
Apparent Temperature (C)    float64
Humidity                    float64
Wind Speed (km/h)           float64
Wind Bearing (degrees)      float64
Visibility (km)             float64
Loud Cover                  float64
Pressure (millibars)        float64
dtype: object

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for i in df.columns:
    if df[i].dtypes == "object":
        df[i] = encoder.fit_transform(df[i])

In [9]:
df.describe()

,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars)
count,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.0,96453.000000
mean,16.006024,0.121780,11.932678,10.855029,0.734899,10.810640,187.509232,10.347325,0.0,1003.235956
std,4.361497,0.343032,9.551546,10.696847,0.195473,6.913571,107.383428,4.192123,0.0,116.969906
min,0.000000,0.000000,-21.822222,-27.716667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,17.000000,0.000000,4.688889,2.311111,0.600000,5.828200,116.000000,8.339800,0.0,1011.900000
50%,18.000000,0.000000,12.000000,12.000000,0.780000,9.965900,180.000000,10.046400,0.0,1016.450000
75%,19.000000,0.000000,18.838889,18.838889,0.890000,14.135800,290.000000,14.812000,0.0,1021.090000
max,26.000000,2.000000,39.905556,39.344444,1.000000,63.852600,359.000000,16.100000,0.0,1046.380000


In [10]:
df.head()

,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars)
0,19,0,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13
1,19,0,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63
2,17,0,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94
3,19,0,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41
4,17,0,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51


In [11]:
corr_matrix = df.corr()
corr_rel = corr_matrix["Apparent Temperature (C)"].sort_values(ascending=False)

In [12]:
corr_rel

Apparent Temperature (C)    1.000000
Temperature (C)             0.992629
Visibility (km)             0.381718
Summary                     0.142760
Wind Bearing (degrees)      0.029031
Pressure (millibars)       -0.000219
Wind Speed (km/h)          -0.056650
Precip Type                -0.521781
Humidity                   -0.602571
Loud Cover                       NaN
Name: Apparent Temperature (C), dtype: float64

df.drop('WindSpeed9am', inplace=True, axis=1)
df.drop('WindSpeed3pm', inplace=True, axis=1)
df.drop('MinTemp', inplace=True, axis=1)
df.drop('WindGustDir', inplace=True, axis=1)
df.drop('WindDir3pm', inplace=True, axis=1)
df.drop('WindDir9am', inplace=True, axis=1)
df.drop('Location', inplace=True, axis=1)
df.drop('Temp9am', inplace=True, axis=1)
df.drop('Evaporation', inplace=True, axis=1)

In [13]:
df["Loud Cover"].unique()

array([0.])

In [14]:
df.drop("Loud Cover", inplace=True, axis=1)
df.drop("Temperature (C)", inplace=True, axis=1)

In [15]:
import numpy as np
df = df.sample(frac = 1)
X=df.copy()
X.drop("Apparent Temperature (C)", inplace=True, axis=1)
y=df["Apparent Temperature (C)"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Base Feedforward Neural Network I

In [16]:
X_train.shape[1]

7

In [17]:

callback = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
model = Sequential(name="Model-with-All-Features") # Model
model.add(Dense(16, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

##### Step 4 - Compile keras model
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1, validation_split=0.2, callbacks=[callback])

##### Step 5 - Fit keras model on the dataset

Epoch 1/100
6173/6173 [==============================] - 10s 1ms/step - loss: 8.6041 - mean_squared_error: 132.6104 - val_loss: 6.8515 - val_mean_squared_error: 71.5915
Epoch 2/100
6173/6173 [==============================] - 9s 1ms/step - loss: 7.2319 - mean_squared_error: 80.6147 - val_loss: 6.4636 - val_mean_squared_error: 63.8544
Epoch 3/100
6173/6173 [==============================] - 9s 1ms/step - loss: 6.9137 - mean_squared_error: 74.0405 - val_loss: 6.8890 - val_mean_squared_error: 73.5923
Epoch 4/100
6173/6173 [==============================] - 9s 1ms/step - loss: 6.7434 - mean_squared_error: 70.5830 - val_loss: 6.1289 - val_mean_squared_error: 57.8113
Epoch 5/100
6173/6173 [==============================] - 10s 2ms/step - loss: 6.6053 - mean_squared_error: 67.7521 - val_loss: 6.5077 - val_mean_squared_error: 66.4618
Epoch 6/100
6173/6173 [==============================] - 11s 2ms/step - loss: 6.4905 - mean_squared_error: 65.4522 - val_loss: 6.5454 - val_mean_squared_error: 64

Epoch 50/100
6173/6173 [==============================] - 8s 1ms/step - loss: 5.4962 - mean_squared_error: 48.7571 - val_loss: 5.3357 - val_mean_squared_error: 45.3661
Epoch 51/100
6173/6173 [==============================] - 8s 1ms/step - loss: 5.4929 - mean_squared_error: 48.6405 - val_loss: 5.4356 - val_mean_squared_error: 49.2818
Epoch 52/100
6173/6173 [==============================] - 8s 1ms/step - loss: 5.4971 - mean_squared_error: 48.7217 - val_loss: 5.6041 - val_mean_squared_error: 53.0022
Epoch 53/100
6173/6173 [==============================] - 8s 1ms/step - loss: 5.4836 - mean_squared_error: 48.4712 - val_loss: 5.3306 - val_mean_squared_error: 45.0646
Epoch 54/100
6173/6173 [==============================] - 8s 1ms/step - loss: 5.4736 - mean_squared_error: 48.3152 - val_loss: 5.4302 - val_mean_squared_error: 49.3866
Epoch 55/100
6173/6173 [==============================] - 8s 1ms/step - loss: 5.4876 - mean_squared_error: 48.6643 - val_loss: 5.3141 - val_mean_squared_error: 

Epoch 99/100
6173/6173 [==============================] - 9s 1ms/step - loss: 5.2629 - mean_squared_error: 45.1096 - val_loss: 5.1072 - val_mean_squared_error: 42.3315
Epoch 100/100
6173/6173 [==============================] - 9s 1ms/step - loss: 5.2563 - mean_squared_error: 44.8361 - val_loss: 5.0977 - val_mean_squared_error: 41.8035


# FFNN With SCaled Features II

In [18]:
import numpy as np
df = df.sample(frac = 1)
X=df.copy()
X.drop("Apparent Temperature (C)", inplace=True, axis=1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
y=df["Apparent Temperature (C)"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [19]:
X

array([[ 0.45717939, -0.35501138,  0.94694531, ...,  1.28037894,
        -0.08714601,  0.07962811],
       [-0.91850209,  2.56018635,  0.84462872, ...,  1.20587916,
        -1.91141007,  0.246595  ],
       [-2.29418357, -0.35501138, -0.38317033, ...,  1.03825467,
         1.30697578,  0.13605317],
       ...,
       [ 0.68645963, -0.35501138, -1.5086528 , ..., -0.41449099,
         0.86147129,  0.17093412],
       [ 0.68645963, -0.35501138,  0.33304578, ..., -0.19099166,
         1.15335354,  0.06124722],
       [ 0.68645963, -0.35501138,  0.38420408, ...,  0.48881881,
         1.37226523,  0.06475241]])

In [20]:
model2 = Sequential(name="Model-with-All-Features") # Model
model2.add(Dense(16, input_shape=(X_train.shape[1],), activation='relu'))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(1, activation='linear'))

##### Step 4 - Compile keras model
model2.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])
model2.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1, validation_split=0.2)

##### Step 5 - Fit keras model on the dataset

Epoch 1/100
6173/6173 [==============================] - 9s 1ms/step - loss: 5.5127 - mean_squared_error: 50.8768 - val_loss: 5.1268 - val_mean_squared_error: 42.7861
Epoch 2/100
6173/6173 [==============================] - 9s 1ms/step - loss: 5.0046 - mean_squared_error: 41.4194 - val_loss: 4.8929 - val_mean_squared_error: 39.9843
Epoch 3/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.7782 - mean_squared_error: 38.3647 - val_loss: 4.7333 - val_mean_squared_error: 38.0331
Epoch 4/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.6886 - mean_squared_error: 37.2300 - val_loss: 4.6918 - val_mean_squared_error: 37.7326
Epoch 5/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.6223 - mean_squared_error: 36.3922 - val_loss: 4.6124 - val_mean_squared_error: 36.2317
Epoch 6/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.5948 - mean_squared_error: 36.1396 - val_loss: 4.6252 - val_mean_squared_error: 36.757

Epoch 50/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.4400 - mean_squared_error: 33.7404 - val_loss: 4.4718 - val_mean_squared_error: 33.9148
Epoch 51/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.4371 - mean_squared_error: 33.7008 - val_loss: 4.4942 - val_mean_squared_error: 35.1892
Epoch 52/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.4399 - mean_squared_error: 33.7198 - val_loss: 4.4630 - val_mean_squared_error: 34.3352
Epoch 53/100
6173/6173 [==============================] - 8s 1ms/step - loss: 4.4395 - mean_squared_error: 33.7585 - val_loss: 4.4569 - val_mean_squared_error: 34.0546
Epoch 54/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.4374 - mean_squared_error: 33.6639 - val_loss: 4.4667 - val_mean_squared_error: 33.9646
Epoch 55/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.4364 - mean_squared_error: 33.6772 - val_loss: 4.4690 - val_mean_squared_error: 

Epoch 99/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.4101 - mean_squared_error: 33.3620 - val_loss: 4.4587 - val_mean_squared_error: 34.5280
Epoch 100/100
6173/6173 [==============================] - 9s 1ms/step - loss: 4.4080 - mean_squared_error: 33.2884 - val_loss: 4.4443 - val_mean_squared_error: 34.0105


### TryOuts with Other Algorithms - RandomForest